In [20]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk

# Download required NLTK data if not present
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')
    
try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')

In [21]:
df = pd.read_csv("./data/processed/data.csv")

df


,text,label,label_text,subject,message
0,any software just for 15 $ - 99 $ understandin...,1,spam,any software just for 15 $ - 99 $,understanding oem software\nlead me not into t...
1,perspective on ferc regulatory action client c...,0,ham,perspective on ferc regulatory action client c...,"19 th , 2 : 00 pm edt\nperspective on ferc reg..."
2,wanted to try ci 4 lis but thought it was way ...,1,spam,wanted to try ci 4 lis but thought it was way ...,viagra at $ 1 . 12 per dose\nready to boost yo...
3,"enron / hpl actuals for december 11 , 2000 tec...",0,ham,"enron / hpl actuals for december 11 , 2000",teco tap 30 . 000 / enron ; 120 . 000 / hpl ga...
4,looking for cheap high - quality software ? ro...,1,spam,looking for cheap high - quality software ? ro...,"water past also , burn , course . gave country..."
...,...,...,...,...,...
31711,"credit netco start up plan louise ,\nattached ...",0,ham,credit netco start up plan,"louise ,\nattached is the electronic version o..."
31712,"everything you are looking for hello , visit o...",1,spam,everything you are looking for,"hello , visit our amazing\nonline shop !\nnows..."
31713,start date : 2 / 5 / 02 ; hourahead hour : 18 ...,0,ham,start date : 2 / 5 / 02 ; hourahead hour : 18 ;,start date : 2 / 5 / 02 ; hourahead hour : 18 ...
31714,corhshucker daren - - - -\nthe invoices are be...,0,ham,corhshucker,daren - - - -\nthe invoices are being paid by ...


---
Lowercase : (text_label dont need lower methode)

Remove punctuation & special characters


In [22]:
import string
import re 

text_cols = [col for col in df.select_dtypes(include=['object']).columns 
             if col != 'label_text']

for col in text_cols:

    # lowercase 
    df[col] = df[col].str.lower()

    # remove punctuation (vectorized)
    df[col] = df[col].str.replace(f"[{re.escape(string.punctuation)}]", '', regex=True)

    # remove special characters
    df[col] = df[col].str.replace(r'[^a-z0-9\s]', '', regex=True)

    # remove extra whitespace
    df[col] = df[col].str.strip().str.replace(r'\s+', ' ', regex=True)





---
Remove punctuation & special characters

- text processing task for data cleaning

The goal is to take an input string (a sentence, paragraph, or body of text) and create a new string where all characters that are considered punctuation or special characters are omitted, leaving only letters, numbers, and sometimes spaces.

---
- tokenize

- remove stop words

- stemming  

In [23]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
from nltk.stem import SnowballStemmer



cols_to_tokenize = ['text', 'subject', 'message']
stop_words_english = set(stopwords.words("english"))

stemmer = SnowballStemmer("english")



# filter callback
def remove_stopwords(tokens):
    return [word for word in tokens if word not in stop_words_english]


# apply stemming to each token
def stem_tokens(tokens):
    return [stemmer.stem(word) for word in tokens]


for col in cols_to_tokenize:

    df[col] = df[col].astype(str) # covert to string

    new_col_name = f"{col}_tokens" # new col to store tokens

    # Tokenize : apply word_tokenize for each row in col
    df[new_col_name] = df[col].apply(word_tokenize)

    # Remove stop words
    df[new_col_name] = df[new_col_name].apply(remove_stopwords)

    # stemming 
    df[new_col_name] = df[new_col_name].apply(stem_tokens)

    # join - convert list to string (need text to vectorize )
    df[new_col_name] = df[new_col_name].apply(lambda tokens: " ".join(tokens))



df


,text,label,label_text,subject,message,text_tokens,subject_tokens,message_tokens
0,any software just for 15 99 understanding oem ...,1,spam,any software just for 15 99,understanding oem software lead me not into te...,softwar 15 99 understand oem softwar lead temp...,softwar 15 99,understand oem softwar lead temptat find way 3...
1,perspective on ferc regulatory action client c...,0,ham,perspective on ferc regulatory action client c...,19 th 2 00 pm edt perspective on ferc regulato...,perspect ferc regulatori action client conf ca...,perspect ferc regulatori action client conf ca...,19 th 2 00 pm edt perspect ferc regulatori act...
2,wanted to try ci 4 lis but thought it was way ...,1,spam,wanted to try ci 4 lis but thought it was way ...,viagra at 1 12 per dose ready to boost your se...,want tri ci 4 lis thought way expens viagra 1 ...,want tri ci 4 lis thought way expens,viagra 1 12 per dose readi boost sex life posi...
3,enron hpl actuals for december 11 2000 teco ta...,0,ham,enron hpl actuals for december 11 2000,teco tap 30 000 enron 120 000 hpl gas daily ls...,enron hpl actual decemb 11 2000 teco tap 30 00...,enron hpl actual decemb 11 2000,teco tap 30 000 enron 120 000 hpl gas daili ls...
4,looking for cheap high quality software rotate...,1,spam,looking for cheap high quality software rotate...,water past also burn course gave country mass ...,look cheap high qualiti softwar rotat napoleon...,look cheap high qualiti softwar rotat napoleon,water past also burn cours gave countri mass l...
...,...,...,...,...,...,...,...,...
31711,credit netco start up plan louise attached is ...,0,ham,credit netco start up plan,louise attached is the electronic version of w...,credit netco start plan louis attach electron ...,credit netco start plan,louis attach electron version debbi deliv afte...
31712,everything you are looking for hello visit our...,1,spam,everything you are looking for,hello visit our amazing online shop nowspec of...,everyth look hello visit amaz onlin shop nowsp...,everyth look,hello visit amaz onlin shop nowspec cia ls vi ...
31713,start date 2 5 02 hourahead hour 18 start date...,0,ham,start date 2 5 02 hourahead hour 18,start date 2 5 02 hourahead hour 18 no ancilla...,start date 2 5 02 hourahead hour 18 start date...,start date 2 5 02 hourahead hour 18,start date 2 5 02 hourahead hour 18 ancillari ...
31714,corhshucker daren the invoices are being paid ...,0,ham,corhshucker,daren the invoices are being paid by us now ja...,corhshuck daren invoic paid us jame armstrong ...,corhshuck,daren invoic paid us jame armstrong take care ...


In [24]:
df.columns

Index(['text', 'label', 'label_text', 'subject', 'message', 'text_tokens',
       'subject_tokens', 'message_tokens'],
      dtype='object')

In [25]:

cols_to_tokenize = ["text_tokens","subject_tokens","message_tokens"]


df["combined_text"] = (
    df["text_tokens"].astype(str) + " " +
    df["subject_tokens"].astype(str) + " " +
    df["message_tokens"].astype(str)
)

df

,text,label,label_text,subject,message,text_tokens,subject_tokens,message_tokens,combined_text
0,any software just for 15 99 understanding oem ...,1,spam,any software just for 15 99,understanding oem software lead me not into te...,softwar 15 99 understand oem softwar lead temp...,softwar 15 99,understand oem softwar lead temptat find way 3...,softwar 15 99 understand oem softwar lead temp...
1,perspective on ferc regulatory action client c...,0,ham,perspective on ferc regulatory action client c...,19 th 2 00 pm edt perspective on ferc regulato...,perspect ferc regulatori action client conf ca...,perspect ferc regulatori action client conf ca...,19 th 2 00 pm edt perspect ferc regulatori act...,perspect ferc regulatori action client conf ca...
2,wanted to try ci 4 lis but thought it was way ...,1,spam,wanted to try ci 4 lis but thought it was way ...,viagra at 1 12 per dose ready to boost your se...,want tri ci 4 lis thought way expens viagra 1 ...,want tri ci 4 lis thought way expens,viagra 1 12 per dose readi boost sex life posi...,want tri ci 4 lis thought way expens viagra 1 ...
3,enron hpl actuals for december 11 2000 teco ta...,0,ham,enron hpl actuals for december 11 2000,teco tap 30 000 enron 120 000 hpl gas daily ls...,enron hpl actual decemb 11 2000 teco tap 30 00...,enron hpl actual decemb 11 2000,teco tap 30 000 enron 120 000 hpl gas daili ls...,enron hpl actual decemb 11 2000 teco tap 30 00...
4,looking for cheap high quality software rotate...,1,spam,looking for cheap high quality software rotate...,water past also burn course gave country mass ...,look cheap high qualiti softwar rotat napoleon...,look cheap high qualiti softwar rotat napoleon,water past also burn cours gave countri mass l...,look cheap high qualiti softwar rotat napoleon...
...,...,...,...,...,...,...,...,...,...
31711,credit netco start up plan louise attached is ...,0,ham,credit netco start up plan,louise attached is the electronic version of w...,credit netco start plan louis attach electron ...,credit netco start plan,louis attach electron version debbi deliv afte...,credit netco start plan louis attach electron ...
31712,everything you are looking for hello visit our...,1,spam,everything you are looking for,hello visit our amazing online shop nowspec of...,everyth look hello visit amaz onlin shop nowsp...,everyth look,hello visit amaz onlin shop nowspec cia ls vi ...,everyth look hello visit amaz onlin shop nowsp...
31713,start date 2 5 02 hourahead hour 18 start date...,0,ham,start date 2 5 02 hourahead hour 18,start date 2 5 02 hourahead hour 18 no ancilla...,start date 2 5 02 hourahead hour 18 start date...,start date 2 5 02 hourahead hour 18,start date 2 5 02 hourahead hour 18 ancillari ...,start date 2 5 02 hourahead hour 18 start date...
31714,corhshucker daren the invoices are being paid ...,0,ham,corhshucker,daren the invoices are being paid by us now ja...,corhshuck daren invoic paid us jame armstrong ...,corhshuck,daren invoic paid us jame armstrong take care ...,corhshuck daren invoic paid us jame armstrong ...


In [ ]:
training_df = df[['combined_text', 'label']].copy()

# Save to CSV
training_df.to_csv('./data/processed/training_data.csv', index=False)

print(f"Training data saved: {len(training_df)} rows")
print(f"Columns: {training_df.columns.tolist()}")
print(f"\nLabel distribution:")
print(training_df['label'].value_counts())

Training data saved: 31716 rows
Columns: ['combined_text', 'label']

Label distribution:
label
1    16163
0    15553
Name: count, dtype: int64
